**Week 3 Part 2 (Part 1 included)**

**1. Scraping the table data from wikipedia page using BeautifulSoup and creating a dataframe with only columns**

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(URL)
soup = BeautifulSoup(response.text,"html.parser")

table = soup.find('table',{'class':'wikitable sortable'}).tbody

rows = table.find_all('tr')

columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]

df = pd.DataFrame(columns=columns)

In [3]:
df.rename(columns={"Postcode":"PostalCode"},inplace=True)
df.columns

Index(['PostalCode', 'Borough', 'Neighbourhood'], dtype='object')

**2. Reading the row data from table rows and appending it into the df dataframe.**

In [4]:
df.head()

,PostalCode,Borough,Neighbourhood


In [5]:
for i in range(1,len(rows)):
    tds = rows[i].find_all('td')
    if tds[1].text != 'Not assigned':
            if (tds[2].text.replace('\n','') == 'Not assigned'):
                values = [tds[0].text,tds[1].text,tds[1].text]
            else :   
                values = [tds[0].text,tds[1].text,tds[2].text.replace('\n','')]
            df = df.append(pd.Series(values,index=columns),ignore_index=True)
    else:
        values = [td.text.replace('\n','') for td in tds]

In [6]:
df.head()

,PostalCode,Borough,Neighbourhood,Postcode
0,NaN,North York,Parkwoods,M3A
1,NaN,North York,Victoria Village,M4A
2,NaN,Downtown Toronto,Harbourfront,M5A
3,NaN,Downtown Toronto,Regent Park,M5A
4,NaN,North York,Lawrence Heights,M6A


**3. Deleting junk column 'PostalCode', renaming PostCode column to 'PostalCode' and re-positioning the column position**

In [7]:
df.drop(['PostalCode'],axis=1,inplace=True)

In [8]:
df.rename(columns={"Postcode":"PostalCode"},inplace=True)
df.columns

Index(['Borough', 'Neighbourhood', 'PostalCode'], dtype='object')

In [9]:
df = df[['PostalCode','Borough','Neighbourhood']]
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


**4. Groupby Postcode and Borough and then join by Neighborhood so that there is only row per Postcode and Borough**

In [10]:
df_grouped = df.groupby(['PostalCode','Borough'],as_index=False).agg(lambda group: ','.join(group))

In [13]:
df_grouped.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
df_grouped.shape

(103, 3)

**5. Reading the Toronto PostalCode coordinates form the csv file**

In [15]:
toronto_coordinates = pd.read_csv('C:/Users/dsapre/Documents/GitHub-Projects/MyGitHub-Example/Geospatial_Coordinates.csv')

In [16]:
toronto_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**6. Renaming Postal Code to PostalCode for merging df_grouped and toronto_coordinates dataframes**

In [17]:
toronto_coordinates.rename(columns={"Postal Code":"PostalCode"},inplace=True)
toronto_coordinates.columns

Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')

In [18]:
df_merge = pd.merge(df_grouped,toronto_coordinates, on = 'PostalCode')

In [19]:
df_merge.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
df_merge.shape

(103, 5)